# Практическая работа №6

# Выполнил студент группы ББМО-01-23 Егоров Ю.А.


##Загрузка и создание двух различных моделей

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
#  Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
#  Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])
#   Компиляция модели
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
#   Обучение модели
model1.fit(train_images, train_labels, epochs=5)
#   Сохранение модели
model1.save('/content/drive/MyDrive/aziis/practica_6/mnist_model1.h5')
#   Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])
# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
#  Обучение модели
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)
# Сохранение модели
model2.save('/content/drive/MyDrive/aziis/practica_6/mnist_model2.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8758 - loss: 0.4308
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9642 - loss: 0.1225
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9767 - loss: 0.0796
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9834 - loss: 0.0558
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9882 - loss: 0.0409


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9101 - loss: 0.2964
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9844 - loss: 0.0513
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9906 - loss: 0.0312
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9934 - loss: 0.0198
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9954 - loss: 0.0146


##Реализация атаки FGSM на первую модель

In [3]:
import numpy as np
# Функция для реализации FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  # Применение знака градиента к изображению
  perturbed_image = image + epsilon * np.sign(gradient)
  # Обрезка значений, чтобы они оставались в пределах [0,1]
  perturbed_image = np.clip(perturbed_image, 0, 1)
  return perturbed_image
# Вычисление градиента
def generate_adversarial_example(model, image, label, epsilon):
    #  Превращаем изображение в формат, подходящий для модели
    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))

    # Если label — это one-hot вектор, преобразуем его в индекс
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label)
    label = tf.convert_to_tensor(label)

    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)

    gradient = tape.gradient(loss, image)

    # Применяем FGSM
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())

    #  Убедимся, что adversarial_image имеет правильную форму
    return np.reshape(adversarial_image, (28, 28, 1))

def generate_adversarial_dataset(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        adv_image = generate_adversarial_example(model, images[i], labels[i], epsilon)
        adversarial_images.append(adv_image.reshape(28, 28))

    adversarial_images = np.array(adversarial_images)

    # Проверка формы
    print("Shape of adversarial_images:", adversarial_images.shape)

    return adversarial_images


# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_adversarial_dataset(model1, test_images, test_labels, epsilon)


Shape of adversarial_images: (10000, 28, 28)


## Оценка противоречивых примеров на обеих моделях

In [4]:
# Оценка первой модели на противоречивых примерах
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0995 - loss: 6.4203
Accuracy of model1 on adversarial examples: 0.12759999930858612
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9463 - loss: 0.1759
Accuracy of model2 on adversarial examples from model1: 0.9562000036239624


## Анализ переносимости атак

In [5]:
adversarial_images_model2 = generate_adversarial_dataset(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28,
28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

Shape of adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9040 - loss: 0.2927
Accuracy of model1 on adversarial examples from model2: 0.9258999824523926


#Вывод:
В ходе выполнения работы было проведено исследование воздействия атаки по переносу, при которой противоречивые примеры, сгенерированные для одной модели, использовались для тестирования другой. Для реализации атак применялся метод FGSM (Fast Gradient Sign Method), позволяющий быстро создавать искажённые примеры, способные повлиять на предсказания моделей.

Результаты эксперимента показали следующее:

- Модель 1, на которой были сгенерированы противоречивые примеры, продемонстрировала значительное снижение точности: с 98% до ~10% (точная точность 0.1276). Это свидетельствует о её высокой уязвимости к прямым FGSM-атакам.
- Модель 2, протестированная на противоречивых примерах, созданных для модели 1, показала существенно меньший эффект. Её точность снизилась незначительно, с ~95% до ~94.6% (точная точность 0.9562), что указывает на её устойчивость к переносу атак.

Дополнительно были проведены эксперименты с противоречивыми примерами, созданными для модели 2:

- Модель 1, протестированная на этих примерах, показала небольшое снижение точности до ~90.4% (точная точность 0.9259). Это падение остаётся значительно меньшим, чем при атаке собственных примеров модели 1.

Таким образом, результаты эксперимента демонстрируют:

- Прямые FGSM-атаки существенно снижают точность модели, для которой генерировались примеры (как в случае модели 1).
- Перенос атак на другую модель менее эффективен, особенно если эта модель изначально устойчива к FGSM (как в случае модели 2).
- Устойчивость моделей к переносу атак, даже при небольшом снижении точности, подчёркивает важность разработки методов, направленных на повышение их защищённости.

Эти выводы подтверждают необходимость создания моделей, обладающих устойчивостью как к прямым атакам, так и к атакам, перенесённым с других моделей.